In [1]:
from pathlib import Path
import os
def load_env_vars_from_file(path: Path):
    file = path.read_text()
    for line in file.splitlines():
        if line.startswith("export"):
            var_to_value = line.split(" ")[1].split("=")
            var_ = var_to_value[0]
            value_ = var_to_value[1][1:-1]
            os.environ[var_] = value_
def set_up_prod_credentials():
    file = "../../../../scripts/prod_setup_creds.sh"
    load_env_vars_from_file(Path(file))
set_up_prod_credentials()

In [2]:
from exchange.interface import ExchangeInterface

e = ExchangeInterface(is_test_run=False)

In [34]:
from datetime import datetime
from helpers.types.portfolio import GetFillsRequest
import pytz
start = datetime(2024, 10, 5).astimezone(pytz.UTC)
fills = e.get_fills(GetFillsRequest(min_ts=int(start.timestamp())))

In [35]:
fills.sort(key=lambda x: x.created_time)

In [36]:
from collections import defaultdict
from dataclasses import dataclass

from helpers.types.orders import Side, TradeType

@dataclass
class ProfitData():
    positions: int = 0 
    spent: int = 0 
    made: int = 0 
# From market ticker to position
pd = defaultdict(lambda: ProfitData())
# Had taker fills

for fill in fills:
    if fill.is_taker == True:
        print("taker fill: ", fill.ticker)

    if (fill.side == Side.YES and fill.action == TradeType.BUY) or (fill.side == Side.NO and fill.action == TradeType.SELL):
        pd[fill.ticker].spent += (fill.yes_price) * (fill.count)
        if pd[fill.ticker].positions < 0:
            pd[fill.ticker].made += fill.count * 100
        pd[fill.ticker].positions += fill.count
    else:
        pd[fill.ticker].spent += (fill.no_price) * (fill.count)
        if pd[fill.ticker].positions > 0:
            pd[fill.ticker].made += fill.count * 100
        pd[fill.ticker].positions -= fill.count



taker fill:  BTCMINY-24DEC31-30000
taker fill:  VINE-24
taker fill:  OSCARNOMPIC-25-B
taker fill:  SPACEXCOUNT-24-130
taker fill:  BTCMAXY-24DEC31-80000
taker fill:  RTTERRIFIER3-75
taker fill:  OSCARNOMACTO-25-TC
taker fill:  MORTGAGEDEF-24-Q3-1.75
taker fill:  TROPSTORM-24DEC01-T15
taker fill:  TROPSTORM-24DEC01-T15
taker fill:  TROPSTORM-24DEC01-T15
taker fill:  USTESTSMATH-24-SI
taker fill:  USTESTSMATH-24-NSD
taker fill:  USTESTSMATH-24-SD
taker fill:  GOOGLECEOCHANGE-24
taker fill:  BTCATH-24-DEC31-73265.12
taker fill:  BTCATH-24-DEC31-73265.12
taker fill:  DEBATES24-2
taker fill:  DEBATES24-3
taker fill:  RATECUTCOUNT-24DEC31-T3
taker fill:  RATECUTCOUNT-24DEC31-T5
taker fill:  GTEMP-24-1.02/23
taker fill:  OSCARCOUNTDUNE2-3
taker fill:  OSCARCOUNTDUNE2-4
taker fill:  OSCARCOUNTDUNE2-7
taker fill:  CPIYOY-24SEP-T2.3
taker fill:  INX-24OCT10-B5787
taker fill:  INX-24OCT10-B5787


In [37]:
pd

defaultdict(<function __main__.<lambda>()>,
            {'BTCMINY-24DEC31-30000': ProfitData(positions=-1, spent=89, made=0),
             'VINE-24': ProfitData(positions=-2, spent=164, made=0),
             'OSCARNOMPIC-25-B': ProfitData(positions=-3, spent=120, made=0),
             'SPACEXCOUNT-24-130': ProfitData(positions=-5, spent=285, made=0),
             'BTCMAXY-24DEC31-80000': ProfitData(positions=-10, spent=740, made=0),
             'RTTERRIFIER3-75': ProfitData(positions=-10, spent=110, made=0),
             'OSCARNOMACTO-25-TC': ProfitData(positions=-10, spent=370, made=0),
             'MORTGAGEDEF-24-Q3-1.75': ProfitData(positions=-10, spent=490, made=0),
             'TROPSTORM-24DEC01-T15': ProfitData(positions=-10, spent=258, made=0),
             'USTESTSMATH-24-SI': ProfitData(positions=5, spent=65, made=0),
             'USTESTSMATH-24-NSD': ProfitData(positions=5, spent=110, made=0),
             'USTESTSMATH-24-SD': ProfitData(positions=5, spent=320, made=0),
 

In [40]:

from helpers.types.markets import MarketResult


final_profit = dict()
for ticker, data in pd.items():
    profit = data.made - data.spent
    if data.positions != 0:
        try:
            m = e.get_market(ticker)
            fi = e.get_fills(GetFillsRequest(ticker=ticker))
        except Exception as ex:
            print(ex)
            continue
        fi.sort(key=lambda x: x.created_time)
        # Dont consider this market if there were fills before the start time 
        if len(fi) > 0 and fi[0].created_time < start:
            print('Had fills before start time: ', m.ticker)
            continue
        if m.result == MarketResult.NO:
            if data.positions < 0:
                profit += abs(data.positions) * 100
        elif m.result == MarketResult.YES:
            if data.positions > 0:
                profit += abs(data.positions) * 100
        else:
            # This market has not settled yet
            print('market not settled: ', m.ticker)
            continue 
    final_profit[ticker] = profit


Had fills before start time:  BTCMINY-24DEC31-30000
Had fills before start time:  VINE-24
Had fills before start time:  OSCARNOMPIC-25-B
Had fills before start time:  SPACEXCOUNT-24-130
Had fills before start time:  BTCMAXY-24DEC31-80000
Had fills before start time:  RTTERRIFIER3-75
Had fills before start time:  OSCARNOMACTO-25-TC
Had fills before start time:  MORTGAGEDEF-24-Q3-1.75
Had fills before start time:  TROPSTORM-24DEC01-T15
Had fills before start time:  USTESTSMATH-24-SI
Had fills before start time:  USTESTSMATH-24-NSD
Had fills before start time:  USTESTSMATH-24-SD
Had fills before start time:  GOOGLECEOCHANGE-24
Had fills before start time:  BTCATH-24-DEC31-73265.12
Had fills before start time:  DEBATES24-2
Had fills before start time:  DEBATES24-3
Had fills before start time:  RATECUTCOUNT-24DEC31-T3
Had fills before start time:  RATECUTCOUNT-24DEC31-T5
404 Client Error: Not Found for url: https://trading-api.kalshi.com/trade-api/v2/markets/GTEMP-24-1.02/23
Had fills befor

In [41]:
final_profit

{'HIGHMIA-24OCT05-T84': -400,
 'HIGHAUS-24OCT05-B95.5': -400,
 'HIGHCHI-24OCT06-T75': 1340,
 'CPICOREYOY-24SEP-T3.3': -300,
 'CPICOREYOY-24SEP-T3.2': 860,
 'CPIYOY-24SEP-T2.3': -860,
 'CPI-24SEP-T0.1': -1520,
 'BTCD-24OCT1017-T61999.99': -320,
 'NASDAQ100U-24OCT10-T20199.99': -911,
 'INX-24OCT10-B5787': -480}

In [42]:
sorted(final_profit.items(), key=lambda x: x[1])

[('CPI-24SEP-T0.1', -1520),
 ('NASDAQ100U-24OCT10-T20199.99', -911),
 ('CPIYOY-24SEP-T2.3', -860),
 ('INX-24OCT10-B5787', -480),
 ('HIGHMIA-24OCT05-T84', -400),
 ('HIGHAUS-24OCT05-B95.5', -400),
 ('BTCD-24OCT1017-T61999.99', -320),
 ('CPICOREYOY-24SEP-T3.3', -300),
 ('CPICOREYOY-24SEP-T3.2', 860),
 ('HIGHCHI-24OCT06-T75', 1340)]

In [44]:
fills = e.get_fills(GetFillsRequest(ticker="CPI-24SEP-T0.1"))

In [45]:
fills

[OrderFill(action=<TradeType.BUY: 'buy'>, count=20, created_time=datetime.datetime(2024, 10, 10, 12, 1, 0, 790778, tzinfo=TzInfo(UTC)), is_taker=False, no_price=76, order_id='d514d716-b602-4cea-91dc-c12b6375b996', side=<Side.NO: 'no'>, ticker='CPI-24SEP-T0.1', trade_id='f5344c94-cd96-48b4-8f5e-f7a97381e4ed', yes_price=24)]